In [2]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:14 http://security.ubuntu.com/ubuntu 

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-13 13:15:20--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2023-01-13 13:15:20 (10.1 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Extract Amazon dataset to dataframe 
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [6]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|          5|            0|          0|   N|                Y|
|R1NK26SWS53B8Q|          5|            0|          0|   N|                Y|
|R11LOHEDYJALTN|          5|            0|          0|   N|                Y|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|                Y|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|                Y|
|R2L15IS24CX0LI|          5|            0|          0|   N|                Y|
|R1DJ8976WPWVZU|          3|            0|          0|   N|     

In [8]:
# Filter the data for reviews that have at least twenty votes
popular_df = vine_df.filter(vine_df["total_votes"] >= 20).orderBy("review_id")
popular_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R10007K8VEMB9R|          2|            7|         24|   N|                N|
|R1000BM31RF4HI|          2|           26|         27|   N|                Y|
|R1003L7NMYTNMM|          2|           14|         24|   N|                N|
|R1004ZWTYXVAB2|          5|           51|         51|   N|                Y|
|R1007NJM6VQZQ1|          5|           62|         67|   N|                Y|
|R100EPX8JZ6D1I|          1|           53|         57|   N|                N|
|R100F6NRV8DEYW|          5|           34|         37|   N|                Y|
|R100G36GGWRVFT|          4|          172|        203|   Y|                N|
|R100JUMKB8134K|          3|            0|         29|   N|                N|
|R100S5TT6TWM1L|          5|          147|        179|   N|     

In [9]:
# Filter the results for reviews where at least half of the votes mark them as helpful
helpful_df = popular_df.filter((popular_df["helpful_votes"] / popular_df["total_votes"]) >= 0.5).orderBy("review_id")
helpful_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1000BM31RF4HI|          2|           26|         27|   N|                Y|
|R1003L7NMYTNMM|          2|           14|         24|   N|                N|
|R1004ZWTYXVAB2|          5|           51|         51|   N|                Y|
|R1007NJM6VQZQ1|          5|           62|         67|   N|                Y|
|R100EPX8JZ6D1I|          1|           53|         57|   N|                N|
|R100F6NRV8DEYW|          5|           34|         37|   N|                Y|
|R100G36GGWRVFT|          4|          172|        203|   Y|                N|
|R100S5TT6TWM1L|          5|          147|        179|   N|                N|
|R100TAAY81EYCN|          1|           57|         68|   N|                Y|
|R100WNDC0PNHW6|          1|           14|         20|   N|     

In [10]:
paid_df = helpful_df.filter(helpful_df["vine"] == "Y").orderBy("review_id")
paid_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100G36GGWRVFT|          4|          172|        203|   Y|                N|
|R1021QF1UB5UAI|          3|           16|         21|   Y|                N|
| R106NP7AO6JJF|          3|           90|        110|   Y|                N|
|R10A2Q2RZDEX4P|          4|           61|         69|   Y|                N|
|R11ZZK2IY6E3MI|          5|           29|         32|   Y|                N|
|R125IZL6QF52HC|          3|           24|         27|   Y|                N|
|R128V68NQDD0HC|          4|          187|        193|   Y|                N|
|R12OU5CDARPZSN|          3|          326|        341|   Y|                N|
|R132B9BM6NF5HV|          2|          106|        110|   Y|                N|
|R13NLMRBSI0F8Y|          4|           25|         28|   Y|     

In [11]:
unpaid_df = helpful_df.filter(helpful_df["vine"] == "N").orderBy("review_id")
unpaid_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1000BM31RF4HI|          2|           26|         27|   N|                Y|
|R1003L7NMYTNMM|          2|           14|         24|   N|                N|
|R1004ZWTYXVAB2|          5|           51|         51|   N|                Y|
|R1007NJM6VQZQ1|          5|           62|         67|   N|                Y|
|R100EPX8JZ6D1I|          1|           53|         57|   N|                N|
|R100F6NRV8DEYW|          5|           34|         37|   N|                Y|
|R100S5TT6TWM1L|          5|          147|        179|   N|                N|
|R100TAAY81EYCN|          1|           57|         68|   N|                Y|
|R100WNDC0PNHW6|          1|           14|         20|   N|                N|
|R1015NB5NVNJ26|          5|           23|         24|   N|     

In [ ]:
# Display total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for all review (based on votes >= 50%)

In [14]:
total_count = helpful_df.count()
total_count

65581

In [15]:
paid_count = paid_df.count()
paid_count

613

In [16]:
unpaid_count = unpaid_df.count()
unpaid_count

64968

In [17]:
five_star_count = helpful_df.filter(helpful_df["star_rating"] == 5).count()
five_star_count

30765

In [18]:
five_star_paid = paid_df.filter(paid_df["star_rating"] == 5).count()
five_star_paid

222

In [19]:
five_star_unpaid = unpaid_df.filter(unpaid_df["star_rating"] == 5).count()
five_star_unpaid

30543

In [22]:
five_star_percentage = five_star_count / total_count * 100
five_star_percentage

46.911453012305394

In [24]:
paid_percentage = five_star_paid / paid_count * 100
paid_percentage

36.215334420880914

In [25]:
unpaid_percentage = five_star_unpaid / unpaid_count * 100
unpaid_percentage

47.01237532323606